## Input Description

Job Machine
3   3
(product 1 #)
(Job) VG  M1      M2      M3
1     1   1   10  2   30  3   20
2     2   1   12  2   13  3   15
3     3   1   21  2   10  3   11

[0, 1, 2, 3, [1, 2], [1, 3], [1, 4], [2, 1], [2, 3], [2, 4], [3, 1], [3, 2], [3, 4], [4, 1], [4, 2], [4, 3]]

population: 3
OS: [1,2,0,2,1,0]
MS: [4,5,6,2,0,1]
 

3 5
1 1 10 2 30 3 20
2 1 12 5 13 3 21 4 28
3 1 21 2 10 3 11

In [1]:
import random

def readFile(filePath):
    with open("./test_data.txt") as fp:
        line = fp.readline()
        machineCnt = list(map(int, line.split()[0:2]))[1]
        nm = line.strip().split(' ') # first line : Job Machine
        line = fp.readline()
        cnt = 0
        jobs = []
        machines = []
        processing_time = []
        VG = []

        while line:
            parse_line = ' '.join(line.split())
            raw_line = parse_line.strip().split(' ')
            i = 1
            jobs.append(cnt)
            cnt += 1
            machine = []
            times = []

            while i < len(raw_line):
                if i % 2 == 0: # 짝수 일 때,
                    m = raw_line[i]
                    times.append(int(m))
                else:
                    m = raw_line[i]
                    machine.append(int(m))
                i += 1
            machines.append(machine)
            processing_time.append(times)
            VG.append(int(raw_line[0]))
            line = fp.readline()
    jobLists = []
    for i in range(len(machines)):
        jobs_dict = []
        for j in range(len(machines[i])):
            jobs_dict.append({'machine':machines[i][j], 'processingTime':processing_time[i][j]})
        jobLists.append(jobs_dict)

    return jobs, VG, machines, processing_time, {'machineCnt': machineCnt, 'jobs': jobLists}
        

In [2]:
jobs, VG, machines, processing_time, jobDicts = readFile("./test_data.txt")

In [3]:
jobDicts

{'machineCnt': 5,
 'jobs': [[{'machine': 1, 'processingTime': 10},
   {'machine': 2, 'processingTime': 30},
   {'machine': 3, 'processingTime': 20}],
  [{'machine': 1, 'processingTime': 12},
   {'machine': 5, 'processingTime': 13},
   {'machine': 3, 'processingTime': 21},
   {'machine': 4, 'processingTime': 28}],
  [{'machine': 1, 'processingTime': 21},
   {'machine': 2, 'processingTime': 10},
   {'machine': 3, 'processingTime': 11}]]}

### permutations

In [127]:
def perm_table(VG, machines):
    VG_tables = []
    k = 0
    for i in machines:
        cnt = []
        per_list = []
        for j in i:
            cnt.append(j)
        for j in permutations(i, VG[k]):
            if(len(i) == VG[k]) or VG[k]==1:
                continue;
            per_list.append(list(j))
        if(len(i) == VG[k]) or VG[k]==1:
            per_list[:0] = cnt
        VG_tables.append(per_list)
        k += 1
    return VG_tables


In [131]:
perm_table = perm_table(VG, machines) # [1, 2, 3]

### generateNM

In [4]:
def generateN(jobs, VG):
    N = []
    job_n = 0
    for v in VG:
        for i in range(v):
            N.append(job_n)
        job_n += 1

    random.shuffle(N)
    return N

In [5]:
def generateM(machines, VG):
    M = []
    cnt = 0

    for v in VG:
        for i in range(v):
            random.shuffle(machines[i])
            M.append(machines[i][v-1])
    
    return M

In [6]:
def initial_generation(jobs, machines, VG):
    pop_gen = []
    for i in range(10):
        NS = generateN(jobs, VG)
        MS = generateM(machines, VG)
        pop_gen.append((NS, MS))

    return pop_gen


In [7]:
parameter = initial_generation(jobs,machines,VG) # NS MS
parameter

[([2, 1, 1, 2, 0, 2], [1, 3, 1, 1, 4, 1]),
 ([2, 2, 1, 0, 2, 1], [1, 2, 1, 3, 3, 3]),
 ([1, 0, 2, 1, 2, 2], [2, 2, 5, 1, 3, 2]),
 ([0, 1, 1, 2, 2, 2], [3, 1, 1, 1, 5, 1]),
 ([2, 2, 2, 1, 0, 1], [1, 2, 3, 1, 3, 2]),
 ([2, 2, 0, 2, 1, 1], [2, 1, 3, 2, 5, 2]),
 ([2, 2, 0, 2, 1, 1], [2, 1, 4, 3, 4, 1]),
 ([2, 2, 1, 0, 1, 2], [1, 3, 3, 3, 4, 2]),
 ([1, 2, 2, 1, 0, 2], [2, 3, 5, 1, 3, 3]),
 ([1, 0, 2, 1, 2, 2], [1, 1, 5, 2, 4, 1])]

In [8]:
def ms_split(VG, param):
    job_lists = []
    current = 0
    (n, m) = param # ([0, 2, 1, 1, 2, 2], [3, 1, 5, 2, 1, 1])
    for v in VG:
        append_l = []
        for i in range(v):
            append_l.append(m[current])
            current += 1
        job_lists.append(append_l)
    return job_lists

In [16]:
def is_free(tab, start, duration):
    for k in range(start, start+duration):
        if not tab[k]:
            return False
    return True

def find_first_available_place(start_ctr, duration, machine_jobs):
    max_duration_list = []
    max_duration = start_ctr + duration

    # max_duration is either the start_ctr + duration or the max(possible starts) + duration
    if machine_jobs:
        for job in machine_jobs:
            max_duration_list.append(job[3] + job[1])  # start + process time

        max_duration = max(max(max_duration_list), start_ctr) + duration

    machine_used = [True] * max_duration

    # Updating array with used places
    for job in machine_jobs:
        start = job[3]
        long = job[1]
        for k in range(start, start + long):
            machine_used[k] = False

    # Find the first available place that meets constraint
    for k in range(start_ctr, len(machine_used)):
        if is_free(machine_used, k, duration):
            return k

In [94]:
def calculateMakespan(mo):
    # Getting the max for each machine
    max_per_machine = []
    for machine in mo:
        max_d = 0
        for job in machine:
            end = job[3] + job[1]
            if end > max_d:
                max_d = end
        max_per_machine.append(max_d)
    return max(max_per_machine)

In [95]:
o = jobDicts['jobs']
sortedPop = sorted(parameter, key=lambda cpl: parameter)
(n, m) = ([0, 2, 1, 1, 2, 2], [3, 1, 5, 2, 1, 1])
cnt = 0

machine_operations = [[] for i in range(jobDicts['machineCnt'])]
ms_s = ms_split(VG, (n, m))
indexes = [0] * len(ms_s)
start_task_cstr = [0] * len(ms_s)

for job in n:
    index_machine = ms_s[job][indexes[job]]
    machine = o[job][indexes[job]]['machine']
    prcTime = o[job][indexes[job]]['processingTime']
    start_cstr = start_task_cstr[job]
    if(len(machine_operations[machine - 1])==0):
        start = 0
    else:
        start = find_first_available_place(start_cstr, prcTime, machine_operations[machine - 1])
    name_task = "{}-{}".format(job, indexes[job]+1)

    machine_operations[machine - 1].append((name_task, prcTime, start, machine))

    indexes[job] += 1
    start_task_cstr[job] = (start + prcTime)
    
mo = machine_operations
calculateMakespan(mo)


        

22